In [2]:
#Ideas
#types of steps, length of steps, types of ingredients length of ingredients


In [3]:
import pandas as pd
import numpy as np

CSV_LEN = 6

def read_data(file):
    df = pd.read_csv(file) 
    X = []
    y = []
    if df.shape[1] == CSV_LEN:
        y = df.loc[:, 'duration_label']
        X = df.iloc[:,:CSV_LEN-1]
    else:
        X = df.iloc[:,:CSV_LEN-1]
    return X, y

In [4]:
X, y = read_data('recipe_train.csv')


In [5]:
from sklearn.tree import DecisionTreeClassifier

class model1:
    def __init__(self):
        self.model =  DecisionTreeClassifier(random_state=0)
        
    def fit(self,X,y):
        mod_X = np.array([X.loc[:,'n_steps']*X.loc[:,'n_ingredients']]).transpose()
        self.model.fit(mod_X,y)
        
    def predict(self,x):
        mod_x = x['n_steps']*x['n_ingredients']
        return self.model.predict(mod_x,y)
        
    def score(self,X,y):
        mod_X = np.array([X.loc[:,'n_steps']*X.loc[:,'n_ingredients']]).transpose()
        return self.model.score(mod_X,y)
        
    

In [6]:
model = model1()
model.fit(X,y)
model.score(X,y)

0.641

In [7]:
from sklearn.tree import DecisionTreeClassifier

class model2:
    def __init__(self):
        self.model =  DecisionTreeClassifier(random_state=0)
        
    def fit(self,X,y):
        mod_X = X.loc[:,['n_steps','n_ingredients']]
        self.model.fit(mod_X,y)
        
    def predict(self,x):
        mod_x = x[['n_steps','n_ingredients']]
        return self.model.predict(mod_x,y)
        
    def score(self,X,y):
        mod_X = X.loc[:,['n_steps','n_ingredients']]
        return self.model.score(mod_X,y)

In [8]:
model = model2()
model.fit(X,y)
model.score(X,y)

0.644125

In [52]:
genmodel = gensim.models.doc2vec.Doc2Vec(vector_size=3, min_count=1, epochs=40)
corpus_namet = X.iloc[[0,100]]['name']
corpus_name=list(tokenize_corpus(corpus_namet))
genmodel.build_vocab(corpus_name)
genmodel.train(corpus_name, total_examples=genmodel.corpus_count, epochs=genmodel.epochs)
genmodel.infer_vector(X.iloc[0])

TypeError: Parameter doc_words of infer_vector() must be a list of strings (not a single string).

In [36]:
#doc2vec provided code
import gensim

# function to preprocess and tokenize text
def tokenize_corpus(txt, tokens_only=False):
    for i, line in enumerate(txt):
        tokens = gensim.utils.simple_preprocess(line)
        if tokens_only:
            yield tokens
        else:
            yield gensim.models.doc2vec.TaggedDocument(tokens, [i])

In [98]:
#doc2vec provided code
import gensim
from sklearn.tree import DecisionTreeClassifier
    
class model3:
    def __init__(self):
        self.vec_size = 5
        self.dtmodel =  DecisionTreeClassifier(random_state=0)
        self.genmodel = gensim.models.doc2vec.Doc2Vec(vector_size=self.vec_size, min_count=1, epochs=40)
        
    def fit(self,X,y):
        train_corpus_name = X['name']
        test_name = X['name']

        # tokenize a training corpus
        corpus_name = list(tokenize_corpus(train_corpus_name))
        
        # train doc2vec on the training corpus
        self.genmodel.build_vocab(corpus_name)
        self.genmodel.train(corpus_name, total_examples=self.genmodel.corpus_count, epochs=self.genmodel.epochs)

        # tokenize new documents
        doc = list(tokenize_corpus(test_name, tokens_only=True))
        # generate embeddings for the new documents
        x_test_name = np.zeros((len(doc),self.vec_size))
        for i in range(len(doc)):
            x_test_name[i,:] = self.genmodel.infer_vector(doc[i])
        self.dtmodel.fit(x_test_name,y)
        
    def predict(self,x):
        doc = list(tokenize_corpus(x['name'], tokens_only=True))
        x_test_name = np.zeros((1,self.vec_size))
        x_test_name[0,:] = self.genmodel.infer_vector(doc[0])
        return self.dtmodel.predict(x_test_name)[0]
        
    def score(self,X,y):
        doc = list(tokenize_corpus(X['name'], tokens_only=True))
        x_test_name = np.zeros((len(doc),self.vec_size))
        for i in range(len(doc)):
            x_test_name[i,:] = self.genmodel.infer_vector(doc[i])
        return self.dtmodel.score(x_test_name,y)
    
model =  model3()
model.fit(X,y)
model.score(X,y)

0.498225

In [27]:
def write_output(model):
    test = pd.read_csv(recipie_test.csv) 
    X, y = read_data(test)
    f = open('output.csv', 'w')
    writer = csv.writer(f)
    writer.writerow(['id','duration_label'])
    for i, row in enumerate(X):
        writer.writerow([i,model.predict(X)[0]])
    f.close()
    


In [34]:
tokenize_corpus(X.loc[0]['name'], tokens_only=True)

<generator object tokenize_corpus at 0x1247e8b10>